<a href="https://colab.research.google.com/github/yutaro-tanaka-yt2705/ag-cgan/blob/main/1_prepare_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import gzip
import sys
import warnings
warnings.filterwarnings("ignore")

## Prepare intake data

## format data

In [ ]:
#read in variant file - it's quicker this way, rather than the pandas read_csv function
filename = '/content/drive/MyDrive/artificial_genome_project/chrX_vcf_formatted.csv.gz'
id_variant_list = []
with gzip.open(filename, "rb") as fi:
  for l in tqdm(fi):
    id_variant_list.append(l.strip().decode("utf-8").split(','))#str(l).replace("b'", '').split(','))
id_variant_df = pd.DataFrame(id_variant_list[1:], columns = id_variant_list[0])
id_variant_df.head()

5097it [00:21, 240.31it/s]


,ID,pair,X_12583_C_T,X_13587_T_C,X_13590_T_C,X_13615_A_G,X_13878_G_C,X_13882_A_C,X_13925_C_A,X_13938_A_G,...,X_156025330_C_T,X_156025332_A_G,X_156025353_C_T,X_156025359_C_T,X_156028989_C_G,X_156029373_T_C,X_156029383_C_G,X_156030556_A_AG,X_156030574_A_AG,X_156030592_A_AG
0,HG00096,A,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,HG00096,B,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,HG00097,A,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,HG00097,B,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,HG00099,A,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
#delete list variable - save space
del id_variant_list

In [ ]:
filtered_variant_info_df = pd.read_csv('/content/drive/MyDrive/artificial_genome_project/chrX_variant_info_filtered_formatted.csv')
filtered_variant_list = filtered_variant_info_df['variant_code'].tolist()

#filter for 1000 variants used to train.
id_variant_df = id_variant_df[['ID', 'pair'] + filtered_variant_list]

In [ ]:
id_variant_df.to_csv('/content/drive/MyDrive/artificial_genome_project/chrX_vcf_1000var_formatted.csv', index=False)

---

## prepare data

In [3]:
id_variant_df = pd.read_csv('/content/drive/MyDrive/artificial_genome_project/chrX_vcf_1000var_formatted.csv')
id_variant_df.head()

,ID,pair,X_1343510_CA_C,X_2464340_G_A,X_844565_T_G,X_1443587_C_T,X_846946_A_G,X_155783945_C_T,X_2348441_C_T,X_1495791_C_G,...,X_484011_A_G,X_1494541_A_G,X_1494970_G_A,X_2272724_A_C,X_2441570_T_C,X_1420810_T_C,X_719380_A_G,X_1115457_A_G,X_2449045_T_C,X_1165297_A_G
0,HG00096,A,0,0,0,1,0,1,1,1,...,0,0,0,1,0,0,0,0,0,1
1,HG00096,B,0,0,1,1,1,1,0,1,...,0,0,0,1,0,0,0,1,0,0
2,HG00097,A,0,0,1,1,1,1,0,1,...,0,0,0,0,0,1,0,0,0,0
3,HG00097,B,0,0,1,1,0,1,0,0,...,0,0,0,1,0,0,0,0,1,0
4,HG00099,A,0,0,1,1,1,1,0,0,...,0,0,0,1,1,1,0,0,1,0


In [30]:
#read in target file
target_file = pd.read_csv('/content/drive/MyDrive/artificial_genome_project/people_info_formatted.csv')

#encode population codes, and save as different dataframe.
from sklearn.preprocessing import OrdinalEncoder
ord_enc = OrdinalEncoder()
target_file["population_code"] = ord_enc.fit_transform(target_file[['Population code']])

population_code_df = target_file[['population_code', 'Population code']].drop_duplicates()

#filter out unnecessary columns
target_file = target_file[['Sample name', 'population_code']]
target_file.columns = ['ID', 'population_code']
target_file.head()

,Sample name,Population code,Population name,Superpopulation code,Superpopulation name,population_code
0,HG00315,FIN,Finnish,EUR,European Ancestry,9.0
1,HG00327,FIN,Finnish,EUR,European Ancestry,9.0
2,HG00334,FIN,Finnish,EUR,European Ancestry,9.0
3,HG00339,FIN,Finnish,EUR,European Ancestry,9.0
4,HG00341,FIN,Finnish,EUR,European Ancestry,9.0


In [5]:
#check population code dataframe
population_code_df.head()

,population_code,Population code
0,9.0,FIN
13,6.0,CHS
15,10.0,GBR
68,22.0,PUR
70,3.0,CDX


In [6]:
#merge target and VCF
id_variant_df = id_variant_df.merge(target_file, on='ID')
id_variant_df.head()

,ID,pair,X_1343510_CA_C,X_2464340_G_A,X_844565_T_G,X_1443587_C_T,X_846946_A_G,X_155783945_C_T,X_2348441_C_T,X_1495791_C_G,...,X_1494541_A_G,X_1494970_G_A,X_2272724_A_C,X_2441570_T_C,X_1420810_T_C,X_719380_A_G,X_1115457_A_G,X_2449045_T_C,X_1165297_A_G,population_code
0,HG00096,A,0,0,0,1,0,1,1,1,...,0,0,1,0,0,0,0,0,1,10.0
1,HG00096,B,0,0,1,1,1,1,0,1,...,0,0,1,0,0,0,1,0,0,10.0
2,HG00097,A,0,0,1,1,1,1,0,1,...,0,0,0,0,1,0,0,0,0,10.0
3,HG00097,B,0,0,1,1,0,1,0,0,...,0,0,1,0,0,0,0,1,0,10.0
4,HG00099,A,0,0,1,1,1,1,0,0,...,0,0,1,1,1,0,0,1,0,10.0


In [28]:
id_variant_df.to_csv('/content/drive/MyDrive/artificial_genome_project/test_dataset.csv', index=False)